# Semiclassical 

In [1]:
import scipy.sparse as sp
import numpy as np
import math
import json
from collections import ChainMap
import itertools
from scipy.sparse import diags
from IPython.display import Markdown
from scipy.sparse.linalg import expm_multiply
#from qiskit import *
import scipy 
from scipy.linalg import expm, sinm, cosm, logm
import copy
import sys
from numpy.linalg import inv
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")
from numpy import linalg as LA
import matplotlib.pyplot as plt

In [2]:
import numpy as np
import copy
from scipy.linalg import eigh
from scipy.sparse.linalg import expm, expm_multiply
from scipy.sparse import csc_matrix, csr_matrix, lil_matrix, eye
import datetime
import os
import argparse


In [3]:
class Hamiltonian_Simulation:
     
    
    def __init__(self,nx,omega,ga,gb, omega_a=4.403989664625457e-05,  T=1.5):
                
        self.omega=omega
        self.omega_b=48*omega_a
        self.ga=ga
        self.gb=gb*1.4
        self.number_of_oscillations=T
        self.trotter_steps=300
        self.tf=self.number_of_oscillations*2*np.pi/omega
        self.dt=self.tf/self.trotter_steps
        self.n=np.round(self.tf/self.dt)
        self.tt=np.linspace(0,self.tf,int(self.n))
        self.dt=self.tt[1]-self.tt[0]
        self.nx=nx
                
        #Position operator 
        self.Lx=(2**nx-1)*(2*np.pi/2**nx)**.5
        self.Lx0=-self.Lx/2 
        self.dx=self.Lx/(2**nx-1)
        self.x=self.Lx0+self.dx*np.array(range(2**nx))
        self.X=np.diag(self.x)
        
        #Momentum operator 
        
        #self.dpx=(2*pi)/(2**self.nx*self.dx)
        #self.px=self.dpx*(0:nx/2)
        #self.px=[self.px(1:end-1),-self.px(end:-1:2)]
               
        diagonals=[1,1*np.ones(2**nx-1),-np.ones(2**nx-1),-1]
        P=-1j/(2*self.dx)*diags(diagonals, [-2**nx+1,1, -1,2**nx-1]).toarray()     
        self.P=P
        
        
        #Paulis Matrices
        self.i= sp.csr_matrix([[1.0,0.0],[0.0,1.0]])
        self.sx = sp.csr_matrix([[0.0,1.0],[1.0,0.0]])
        self.sz = sp.csr_matrix([[1.0,0.0],[0.0,-1.0]])
        self.sy = -1j * (self.sz @ self.sx)
        
        #We create ID
        self.I = sp.eye(2**nx)
        self.Ix = sp.eye(2**nx)
        
        
    #this function given an operator as a dictionary where the keys are the components in Paulis and the 
    #values their coefficients, recreates the operator.     
        
    def operador(self,Dic):
        
        M=len(list(Dic.keys())[0])
        OP=sp.eye(2**M)-sp.eye(2**M)
        
        for i in Dic:
            s=1
            for j in i:       
                if j==0:
                    s1=sp.csr_matrix([[1.0,0.0],[0.0,1.0]])
                if j==1:
                    s1=sp.csr_matrix([[0.0,1.0],[1.0,0.0]])
                if j==2:
                    s1=-1j *(sp.csr_matrix([[1.0,0.0],[0.0,-1.0]])@sp.csr_matrix([[0.0,1.0],[1.0,0.0]]))
                if j==3:
                    s1=sp.csr_matrix([[1.0,0.0],[0.0,-1.0]])
                    
                s=sp.kron(s,s1)
                 
            OP=OP+s*Dic[i]   
        
        return OP
        
    #This function decomposes an operator in Paulis and encodes it in a dictionary.
    
    def decomposition(self,M,D):
        
        S=[self.i, self.sx, self.sy, self.sz]
        a=[]
        index=[0,1,2,3]
        for i in range(D):
            a.append(index)
        a=list(itertools.product(*a))
        Plist=[]
        for i in range(len(a)):
            P=S[a[i][0]]
            for j in range(D-1):
                P=sp.kron(P,S[a[i][j+1]])
            #if (1/2**D*(P@M).diagonal().sum())>10**(-5):
            #print(1/2**D*(P@M).diagonal().sum())
            #print(P.shape[0],M.shape[0])
            Plist.append(1/2**D*(P@M).diagonal().sum())
        
        Dic=dict(zip(a, Plist))
        
        
        for i in list(Dic.keys()):        
            
            if abs(Dic[i])<10**(-8): #or i==a[0]:
                del(Dic[i])
        return Dic       
    
    def Hamiltonian_1(self):
        


        H=Hamiltonian_Simulation(self.nx, self.omega, self.ga, self.gb).decomposition(self.P,self.nx)
        H_P_old=H
        H_P={}
        for key in list(H.keys()):
            new_key=[]
            
            for index in range(len(key)):
                if key[index]==0:
                    new_key.append("Id")
                if key[index]==1:
                    new_key.append("Sx")
                if key[index]==2:
                    new_key.append("Sy")
                if key[index]==3:
                    new_key.append("Sz")
            H_P[tuple(new_key)]=np.real(H[key])
        
        H=Hamiltonian_Simulation(self.nx, self.omega, self.ga, self.gb).decomposition(self.X,self.nx)
        
        H_X={}
        H_X_old=H
        for key in list(H.keys()):
            new_key=[]
            
            for index in range(len(key)):
                if key[index]==0:
                    new_key.append("Id")
                if key[index]==1:
                    new_key.append("Sy")
                if key[index]==2:
                    new_key.append("Sy")
                if key[index]==3:
                    new_key.append("Sz")
            H_X[tuple(new_key)]=np.real(H[key])
            
            
        H_XP={}
        for key in H_X.keys():

            key_list = list(key)

            for i in range(n):

                    key_list[i] = f"{key_list[i]}_{i+2}"

            for key_2 in H_P.keys():

                key_list_2 = list(key_2)

                for i in range(n):

                    key_list_2[i] = f"{key_list_2[i]}_{i+n+2}"

                key_new = ["Id_0", "Id_1"]+key_list+key_list_2
                key_new = tuple(key_new)

                H_XP[key_new]=-np.round(H_X[key]*H_P[key_2],3)*self.omega/2


        H_PX={}
        for key in H_P.keys():

            key_list = list(key)

            for i in range(n):

                    key_list[i] = f"{key_list[i]}_{i+2}"

            for key_2 in H_X.keys():

                key_list_2 = list(key_2)

                for i in range(n):

                    key_list_2[i] = f"{key_list_2[i]}_{i+n+2}"

                key_new = ["Id_0", "Id_1"]+key_list+key_list_2
                key_new = tuple(key_new)

                H_PX[key_new]=np.round(H_X[key_2]*H_P[key],3)*self.omega/2



        H_HO=dict(ChainMap(H_PX, H_XP))


        
        H_int_spin_1={}
        
        for key in H_P.keys():

            key_list = list(key)

            for i in range(n):

                    key_list[i] = f"{key_list[i]}_{i+n+2}"

            aux=[f'Id_{i}' for i in range(2,2+n)]
            key_new = ["Sz_0", "Id_1"]+aux+key_list
            key_new = tuple(key_new)

            H_int_spin_1[key_new]=np.round(-2*np.round(H_P[key],3)*self.ga*2**(1/2),3)
            
        
        H_int_spin_2={}
        
        for key in H_P.keys():

            key_list = list(key)

            for i in range(n):

                    key_list[i] = f"{key_list[i]}_{i+n+2}"

            aux=[f'Id_{i}' for i in range(2,2+n)]
            key_new = ["Id_0", "Sz_1"]+aux+key_list
            key_new = tuple(key_new)

            H_int_spin_2[key_new]=np.round(-2*np.round(H_P[key],3)*self.gb*2**(1/2),3)

        
        H_int=dict(ChainMap(H_int_spin_1, H_int_spin_2))
       
        
        json_str = json.dumps({str(key): value for key, value in H_HO.items()}, indent=4)

        # Parse and print the JSON-like string
        formatted_data = json.loads(json_str)
        print("Hamiltonian of the HO term in KvN formalism")
        print("Qubit 0 -> spin 1")
        print("Qubit 1 -> spin 2")
        print(f"Qubits 2-{2+n-1} -> Classical X")
        print(f"Qubits {2+n}-{2+2*n-1} -> Classical P")
        print(json.dumps(formatted_data, indent=4))     
        
        
        
        
        json_str = json.dumps({str(key): value for key, value in H_int.items()}, indent=4)

        # Parse and print the JSON-like string
        formatted_data = json.loads(json_str)
        print(" ")
        print(" ")
        print("Interaction Hamiltonian KvN")
        
        print(json.dumps(formatted_data, indent=4))     
            
            
                   
        return  H_HO,H_int, H_P_old,H_P,H_X_old, H_X
                          
    


In [4]:
#n is the number of qubits 
#omega the frequency of the HO
#ga and gb are the coupling terms
n=4
omega=10
ga=omega
gb=1.4*omega
H_HO,H_int, H_P_old,H_P,H_X_old, H_X=Hamiltonian_Simulation(n, omega, ga, gb).Hamiltonian_1()

Hamiltonian of the HO term in KvN formalism
Qubit 0 -> spin 1
Qubit 1 -> spin 2
Qubits 2-5 -> Classical X
Qubits 6-9 -> Classical P
{
    "('Id_0', 'Id_1', 'Id_2', 'Id_3', 'Id_4', 'Sz_5', 'Id_6', 'Id_7', 'Id_8', 'Sy_9')": 1.25,
    "('Id_0', 'Id_1', 'Id_2', 'Id_3', 'Id_4', 'Sz_5', 'Id_6', 'Id_7', 'Sx_8', 'Sy_9')": -0.625,
    "('Id_0', 'Id_1', 'Id_2', 'Id_3', 'Id_4', 'Sz_5', 'Id_6', 'Id_7', 'Sy_8', 'Sx_9')": 0.625,
    "('Id_0', 'Id_1', 'Id_2', 'Id_3', 'Id_4', 'Sz_5', 'Id_6', 'Sx_7', 'Sx_8', 'Sy_9')": -0.315,
    "('Id_0', 'Id_1', 'Id_2', 'Id_3', 'Id_4', 'Sz_5', 'Id_6', 'Sx_7', 'Sy_8', 'Sx_9')": -0.315,
    "('Id_0', 'Id_1', 'Id_2', 'Id_3', 'Id_4', 'Sz_5', 'Id_6', 'Sy_7', 'Sx_8', 'Sx_9')": 0.315,
    "('Id_0', 'Id_1', 'Id_2', 'Id_3', 'Id_4', 'Sz_5', 'Id_6', 'Sy_7', 'Sy_8', 'Sy_9')": -0.315,
    "('Id_0', 'Id_1', 'Id_2', 'Id_3', 'Id_4', 'Sz_5', 'Sx_6', 'Sx_7', 'Sx_8', 'Sy_9')": -0.315,
    "('Id_0', 'Id_1', 'Id_2', 'Id_3', 'Id_4', 'Sz_5', 'Sx_6', 'Sx_7', 'Sy_8', 'Sx_9')": -0.315,
    "(

In [5]:
def to_latex(input_dict):
    latex_expression = []
    
    for key, value in input_dict.items():
        terms = []
        for term in key:
            if term.startswith('Id'):
                continue
            elif term.startswith('Sz'):
                terms.append(f'\\sigma^z_{term.split("_")[1]}')
            elif term.startswith('Sx'):
                terms.append(f'\\sigma^x_{term.split("_")[1]}')
            elif term.startswith('Sy'):
                terms.append(f'\\sigma^y_{term.split("_")[1]}')
            else:
                raise ValueError(f"Unknown term: {term}")
        
        terms_str = ''.join(terms)
        latex_expression.append(f'{value}({terms_str})')
    
    return ' + '.join(latex_expression)

latex_output = to_latex(H_HO)
latex_output = latex_output.replace('+ -', '-')

latex_output_2 = to_latex(H_int)


Markdown(f" #### Qubit 0 -> spin 1 \n\n #### Qubit 1 -> spin 2 \n\n #### Qubits 2-{2+n-1} -> Classical X \n\n #### Qubits {2+n}-{2+2*n-1} -> Classical P \n\n ### Hamiltonian of the HO term in KvN formalism:\n\n $$\n{latex_output}\n$$ \n \n ### Interaction Hamiltonian KvN:\n\n$$\n{latex_output_2}\n$$")

 #### Qubit 0 -> spin 1 

 #### Qubit 1 -> spin 2 

 #### Qubits 2-5 -> Classical X 

 #### Qubits 6-9 -> Classical P 

 ### Hamiltonian of the HO term in KvN formalism:

 $$
1.25(\sigma^z_5\sigma^y_9) -0.625(\sigma^z_5\sigma^x_8\sigma^y_9) + 0.625(\sigma^z_5\sigma^y_8\sigma^x_9) -0.315(\sigma^z_5\sigma^x_7\sigma^x_8\sigma^y_9) -0.315(\sigma^z_5\sigma^x_7\sigma^y_8\sigma^x_9) + 0.315(\sigma^z_5\sigma^y_7\sigma^x_8\sigma^x_9) -0.315(\sigma^z_5\sigma^y_7\sigma^y_8\sigma^y_9) -0.315(\sigma^z_5\sigma^x_6\sigma^x_7\sigma^x_8\sigma^y_9) -0.315(\sigma^z_5\sigma^x_6\sigma^x_7\sigma^y_8\sigma^x_9) -0.315(\sigma^z_5\sigma^x_6\sigma^y_7\sigma^x_8\sigma^x_9) + 0.315(\sigma^z_5\sigma^x_6\sigma^y_7\sigma^y_8\sigma^y_9) + 2.5(\sigma^z_4\sigma^y_9) -1.25(\sigma^z_4\sigma^x_8\sigma^y_9) + 1.25(\sigma^z_4\sigma^y_8\sigma^x_9) -0.625(\sigma^z_4\sigma^x_7\sigma^x_8\sigma^y_9) -0.625(\sigma^z_4\sigma^x_7\sigma^y_8\sigma^x_9) + 0.625(\sigma^z_4\sigma^y_7\sigma^x_8\sigma^x_9) -0.625(\sigma^z_4\sigma^y_7\sigma^y_8\sigma^y_9) -0.625(\sigma^z_4\sigma^x_6\sigma^x_7\sigma^x_8\sigma^y_9) -0.625(\sigma^z_4\sigma^x_6\sigma^x_7\sigma^y_8\sigma^x_9) -0.625(\sigma^z_4\sigma^x_6\sigma^y_7\sigma^x_8\sigma^x_9) + 0.625(\sigma^z_4\sigma^x_6\sigma^y_7\sigma^y_8\sigma^y_9) + 5.0(\sigma^z_3\sigma^y_9) -2.5(\sigma^z_3\sigma^x_8\sigma^y_9) + 2.5(\sigma^z_3\sigma^y_8\sigma^x_9) -1.25(\sigma^z_3\sigma^x_7\sigma^x_8\sigma^y_9) -1.25(\sigma^z_3\sigma^x_7\sigma^y_8\sigma^x_9) + 1.25(\sigma^z_3\sigma^y_7\sigma^x_8\sigma^x_9) -1.25(\sigma^z_3\sigma^y_7\sigma^y_8\sigma^y_9) -1.25(\sigma^z_3\sigma^x_6\sigma^x_7\sigma^x_8\sigma^y_9) -1.25(\sigma^z_3\sigma^x_6\sigma^x_7\sigma^y_8\sigma^x_9) -1.25(\sigma^z_3\sigma^x_6\sigma^y_7\sigma^x_8\sigma^x_9) + 1.25(\sigma^z_3\sigma^x_6\sigma^y_7\sigma^y_8\sigma^y_9) + 10.0(\sigma^z_2\sigma^y_9) -5.0(\sigma^z_2\sigma^x_8\sigma^y_9) + 5.0(\sigma^z_2\sigma^y_8\sigma^x_9) -2.5(\sigma^z_2\sigma^x_7\sigma^x_8\sigma^y_9) -2.5(\sigma^z_2\sigma^x_7\sigma^y_8\sigma^x_9) + 2.5(\sigma^z_2\sigma^y_7\sigma^x_8\sigma^x_9) -2.5(\sigma^z_2\sigma^y_7\sigma^y_8\sigma^y_9) -2.5(\sigma^z_2\sigma^x_6\sigma^x_7\sigma^x_8\sigma^y_9) -2.5(\sigma^z_2\sigma^x_6\sigma^x_7\sigma^y_8\sigma^x_9) -2.5(\sigma^z_2\sigma^x_6\sigma^y_7\sigma^x_8\sigma^x_9) + 2.5(\sigma^z_2\sigma^x_6\sigma^y_7\sigma^y_8\sigma^y_9) -1.25(\sigma^y_5\sigma^z_9) -2.5(\sigma^y_5\sigma^z_8) -5.0(\sigma^y_5\sigma^z_7) -10.0(\sigma^y_5\sigma^z_6) + 0.625(\sigma^x_4\sigma^y_5\sigma^z_9) + 1.25(\sigma^x_4\sigma^y_5\sigma^z_8) + 2.5(\sigma^x_4\sigma^y_5\sigma^z_7) + 5.0(\sigma^x_4\sigma^y_5\sigma^z_6) -0.625(\sigma^y_4\sigma^x_5\sigma^z_9) -1.25(\sigma^y_4\sigma^x_5\sigma^z_8) -2.5(\sigma^y_4\sigma^x_5\sigma^z_7) -5.0(\sigma^y_4\sigma^x_5\sigma^z_6) + 0.315(\sigma^x_3\sigma^x_4\sigma^y_5\sigma^z_9) + 0.625(\sigma^x_3\sigma^x_4\sigma^y_5\sigma^z_8) + 1.25(\sigma^x_3\sigma^x_4\sigma^y_5\sigma^z_7) + 2.5(\sigma^x_3\sigma^x_4\sigma^y_5\sigma^z_6) + 0.315(\sigma^x_3\sigma^y_4\sigma^x_5\sigma^z_9) + 0.625(\sigma^x_3\sigma^y_4\sigma^x_5\sigma^z_8) + 1.25(\sigma^x_3\sigma^y_4\sigma^x_5\sigma^z_7) + 2.5(\sigma^x_3\sigma^y_4\sigma^x_5\sigma^z_6) -0.315(\sigma^y_3\sigma^x_4\sigma^x_5\sigma^z_9) -0.625(\sigma^y_3\sigma^x_4\sigma^x_5\sigma^z_8) -1.25(\sigma^y_3\sigma^x_4\sigma^x_5\sigma^z_7) -2.5(\sigma^y_3\sigma^x_4\sigma^x_5\sigma^z_6) + 0.315(\sigma^y_3\sigma^y_4\sigma^y_5\sigma^z_9) + 0.625(\sigma^y_3\sigma^y_4\sigma^y_5\sigma^z_8) + 1.25(\sigma^y_3\sigma^y_4\sigma^y_5\sigma^z_7) + 2.5(\sigma^y_3\sigma^y_4\sigma^y_5\sigma^z_6) + 0.315(\sigma^x_2\sigma^x_3\sigma^x_4\sigma^y_5\sigma^z_9) + 0.625(\sigma^x_2\sigma^x_3\sigma^x_4\sigma^y_5\sigma^z_8) + 1.25(\sigma^x_2\sigma^x_3\sigma^x_4\sigma^y_5\sigma^z_7) + 2.5(\sigma^x_2\sigma^x_3\sigma^x_4\sigma^y_5\sigma^z_6) + 0.315(\sigma^x_2\sigma^x_3\sigma^y_4\sigma^x_5\sigma^z_9) + 0.625(\sigma^x_2\sigma^x_3\sigma^y_4\sigma^x_5\sigma^z_8) + 1.25(\sigma^x_2\sigma^x_3\sigma^y_4\sigma^x_5\sigma^z_7) + 2.5(\sigma^x_2\sigma^x_3\sigma^y_4\sigma^x_5\sigma^z_6) + 0.315(\sigma^x_2\sigma^y_3\sigma^x_4\sigma^x_5\sigma^z_9) + 0.625(\sigma^x_2\sigma^y_3\sigma^x_4\sigma^x_5\sigma^z_8) + 1.25(\sigma^x_2\sigma^y_3\sigma^x_4\sigma^x_5\sigma^z_7) + 2.5(\sigma^x_2\sigma^y_3\sigma^x_4\sigma^x_5\sigma^z_6) -0.315(\sigma^x_2\sigma^y_3\sigma^y_4\sigma^y_5\sigma^z_9) -0.625(\sigma^x_2\sigma^y_3\sigma^y_4\sigma^y_5\sigma^z_8) -1.25(\sigma^x_2\sigma^y_3\sigma^y_4\sigma^y_5\sigma^z_7) -2.5(\sigma^x_2\sigma^y_3\sigma^y_4\sigma^y_5\sigma^z_6)
$$ 
 
 ### Interaction Hamiltonian KvN:

$$
-44.239(\sigma^z_1\sigma^y_9) + 22.119(\sigma^z_1\sigma^x_8\sigma^y_9) + -22.119(\sigma^z_1\sigma^y_8\sigma^x_9) + 11.032(\sigma^z_1\sigma^x_7\sigma^x_8\sigma^y_9) + 11.032(\sigma^z_1\sigma^x_7\sigma^y_8\sigma^x_9) + -11.032(\sigma^z_1\sigma^y_7\sigma^x_8\sigma^x_9) + 11.032(\sigma^z_1\sigma^y_7\sigma^y_8\sigma^y_9) + 11.032(\sigma^z_1\sigma^x_6\sigma^x_7\sigma^x_8\sigma^y_9) + 11.032(\sigma^z_1\sigma^x_6\sigma^x_7\sigma^y_8\sigma^x_9) + 11.032(\sigma^z_1\sigma^x_6\sigma^y_7\sigma^x_8\sigma^x_9) + -11.032(\sigma^z_1\sigma^x_6\sigma^y_7\sigma^y_8\sigma^y_9) + -22.571(\sigma^z_0\sigma^y_9) + 11.285(\sigma^z_0\sigma^x_8\sigma^y_9) + -11.285(\sigma^z_0\sigma^y_8\sigma^x_9) + 5.629(\sigma^z_0\sigma^x_7\sigma^x_8\sigma^y_9) + 5.629(\sigma^z_0\sigma^x_7\sigma^y_8\sigma^x_9) + -5.629(\sigma^z_0\sigma^y_7\sigma^x_8\sigma^x_9) + 5.629(\sigma^z_0\sigma^y_7\sigma^y_8\sigma^y_9) + 5.629(\sigma^z_0\sigma^x_6\sigma^x_7\sigma^x_8\sigma^y_9) + 5.629(\sigma^z_0\sigma^x_6\sigma^x_7\sigma^y_8\sigma^x_9) + 5.629(\sigma^z_0\sigma^x_6\sigma^y_7\sigma^x_8\sigma^x_9) + -5.629(\sigma^z_0\sigma^x_6\sigma^y_7\sigma^y_8\sigma^y_9)
$$

In [6]:
def to_matrix(input_dict, num_qubits):
    paulis = np.array([np.array([[1,0],[0,1]]), np.array([[0,1],[1,0]]), np.array([[0,-1.j],[1.j,0]]), np.array([[1,0],[0,-1]])])
    # pauli_tensor = np.array([[np.kron(pauli[i], pauli[j]) for i in range(4)] for j in range(4)])

    def kronecker_pad(paulis, num_qubits, sites):
        kron_list = [np.eye(2) for i in range(num_qubits)]
        for i, site in enumerate(sites):
            kron_list[site] = paulis[i]

        padded_matrix = kron_list[0]
        for i in range(1, len(kron_list)):
            padded_matrix = np.kron(kron_list[i], padded_matrix)
        return padded_matrix
    
    terms = []

    for term in input_dict:
        pauli_list = []
        pauli_inds = []
        for i, pauli in enumerate(term):
            if pauli[1] == 'x':
                pauli_list.append(paulis[1])
                pauli_inds.append(i)
            elif pauli[1] == 'y':
                pauli_list.append(paulis[2])
                pauli_inds.append(i)
            elif pauli[1] == 'z':
                pauli_list.append(paulis[3])
                pauli_inds.append(i)
        terms.append(input_dict[term] * kronecker_pad(pauli_list, num_qubits, pauli_inds))

    return sum(terms)

Ho_matrix = to_matrix(H_HO, (2*n)+2)
H_int_matrix = to_matrix(H_int, (2*n)+2)
H = Ho_matrix + H_int_matrix

In [7]:
def correction_state(ham, energy, state, tau=0.0001):
    #op = expm(-tau*(ham-energy))
    #correction_state = op @ state
    correction_state = ham @ state - energy*state
    return correction_state / np.linalg.norm(correction_state)

# def eff_ham(ham, basis_set):
#     eff_H = np.eye(len(basis_set), dtype=complex)
#     for i in range(len(basis_set)):
#         for j in range(i,len(basis_set)):
#             overlap = basis_set[i].conj().transpose(copy=False).dot(ham.dot(basis_set[j]))[0,0]
#             eff_H[i][j] = overlap
#             if (i != j):
#                 eff_H[j][i] = np.conj(overlap)
#     return eff_H

# def eff_overlap(basis_set):
#     eff_S = np.eye(len(basis_set), dtype=complex)
#     for i in range(len(basis_set)):
#         for j in range(i,len(basis_set)):
#             # overlap = np.dot(basis_set[i], basis_set[j].conj().transpose())
#             overlap = basis_set[i].conj().transpose(copy=False).dot(basis_set[j])[0,0]
#             eff_S[i][j] = overlap
#             if (i != j):
#                 eff_S[j][i] = np.conj(overlap)
#     return eff_S

def eff_ham(ham, basis_set):
    eff_H = np.eye(len(basis_set), dtype=complex)
    for i in range(len(basis_set)):
        for j in range(i,len(basis_set)):
            overlap = np.conj(basis_set[i]).dot(ham.dot(basis_set[j]))
            eff_H[i][j] = overlap
            if (i != j):
                eff_H[j][i] = np.conj(overlap)
    return eff_H

def eff_overlap(basis_set):
    eff_S = np.eye(len(basis_set), dtype=complex)
    for i in range(len(basis_set)):
        for j in range(i,len(basis_set)):
            overlap = basis_set[i].conj().dot(basis_set[j])
            eff_S[i][j] = overlap
            if (i != j):
                eff_S[j][i] = np.conj(overlap)
    return eff_S

def qdavidson_iter(ham, basis_set, tol=0.5):
    num_basis = len(basis_set)
    eff_H = eff_ham(ham, basis_set)
    eff_S = eff_overlap(basis_set)
    evals, evecs = eigh(eff_H, eff_S)
    estates = [np.array(sum([evecs[:,i][j] * basis_set[j] for j in range(num_basis)])) for i in range(num_basis)]
    new_basis_set = copy.deepcopy(basis_set)
    residue_vals = []
    for i in range(num_basis):
        val = np.linalg.norm((ham @ estates[i]) - (evals[i] * estates[i]))
        residue_vals.append(val)
        if val > tol:
            state = correction_state(ham, evals[i], estates[i])
            if linear_independence(state, new_basis_set, eff_S, tol):
                '''
                eff_S = np.pad(eff_S, ((0, 1), (0, 1)), mode='constant')
                for i in range(len(new_basis_set)):
                    overlap = state.conj().T @ new_basis_set[i]
                    eff_S[i][len(new_basis_set)] = overlap
                    eff_S[len(new_basis_set)][i] = overlap
                '''
                new_basis_set.append(state)
                eff_S = eff_overlap(new_basis_set)
                eff_H = eff_ham(ham, new_basis_set)

    return evals, estates, residue_vals, new_basis_set, eff_H, eff_S

def linear_independence(correction_vec, basis_set, eff_S, tol=0.01):
    b = np.array([correction_vec.conj().T.dot(basis_set[i]) for i in range(len(basis_set))])
    if np.all(np.round(eff_S, 8) == np.eye(len(basis_set))):
        return np.linalg.norm(b) < tol
    else:
        return np.linalg.norm(np.linalg.pinv(eff_S).dot(b)) < tol

def qdavidson(ham, initial_basis_set, num_iter, tol=0.5):
    basis_set = copy.deepcopy(initial_basis_set)
    for i in range(num_iter):
        evals, estates, residue_vals, basis_set, eff_H, eff_S = qdavidson_iter(ham, basis_set, tol)
    return evals, estates, residue_vals, basis_set, eff_H, eff_S

def QDFFEvolve(S_inv, H, t, basis, init):
    ct = expm(-1j * t * S_inv @ H) @ init
    return sum([ct[i] * basis[i] for i in range(len(ct))])

In [8]:
epsilon = 0.1  # we want the fidelity with the exact state to be at least this at time = tf

tf = 10
ts = np.linspace(0, tf, 200)

eff_S_eps = 1e-9  # cutoff values in eff_S less than this value

cs = list("01010101010101010101010101010101010101010101010101")
num_qubits = (2*n)+2
use_trotter = 0

In [9]:

c = cs[0:num_qubits]
init = np.zeros(2**num_qubits)
init[int(''.join(c), 2)] = 1
basis_set = [init]
# ham = fast_heisenberg(i)
ham = H
# ham = heisenberg(i, x_hopping_coeff=x_hopping, y_hopping_coeff=y_hopping, z_hopping_coeff=z_hopping, bias_coeff=h)
exact_final_te = expm_multiply(-1j * ham * ts[-1], init)



In [13]:
filename = f"./results/test.txt"
file = open(filename, 'w')
file.write("qubits,num_iters,num_states,fidelity\n")

num_iters = 1
while(True):
    _, _, _, basis_set, eff_H, eff_S = qdavidson(ham, basis_set, 1, tol=0.0001)
    c0 = np.zeros(len(basis_set))
    c0[0] = 1

    qdff_final_te = QDFFEvolve(np.linalg.inv(eff_S), eff_H, ts[-1], basis_set, c0)

    fidelity = np.abs(np.conj(qdff_final_te) @ exact_final_te)**2
    file.write(f"{num_iters},{len(basis_set)},{fidelity}\n")
    if (fidelity > 1-epsilon):
        break

    num_iters += 1
file.close()

: 